In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib inline
import os
import re
import sys

from mbt.tools.faults import rates_for_double_truncated_mfd
from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.mfd import EvenlyDiscretizedMFD

from openquake.hazardlib.scalerel.wc1994 import WC1994

In [ ]:
try:
    plot_figures
except: 
    plot_figures = False
    
try:
    print_log
except: 
    print_log = False

In [ ]:
if print_log or plot_figures:
    print '------------------------------------------------------'
# Width of bins used to discretise the MFDs
model_key = metys.g_prj.models.current 
binwidth = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
if print_log:
    print 'MFD bin width    : %.2f' % (binwidth)

# Load sources
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_fault_sources.pkl' % model_key)
fin = open(data_path,'rb') 
sources = pickle.load(fin)
fin.close()

# Check input variables
if area_source_ids_list is None:
    area_source_ids_list = ['10']
    
if len(area_source_ids_list) > 1:
    assert 0 == 1
    
src = sources[area_source_ids_list[0]]

## Parameters

In [ ]:
default_mmin = 6.5
bin_width = 0.1
scalerel_name = 'WC1994'

## Set the magnitude scaling relationship
The options available for the magnitude scaling relationship are:
* `scalrel_name = 'WC1994`' <br> Wells and Coppersmith (1994) scaling relationship
* '`PointMSR`' Scaling relationship to be used for the creation of tiny ruptures resembling points
* '`CEUS2011`' Scaling relationship developed within the CEUS SSC project

In [ ]:
import importlib
module = importlib.import_module('openquake.hazardlib.scalerel')
my_class = getattr(module, scalerel_name)
scalrel = my_class()

## Computing the mfd

In [ ]:
assert len(source_ids_list) == 1
src = model.sources[source_ids_list[0]]

if src.mmin > 0.0:
    m_low = src.mmin
else:
    m_low = default_mmin

for key in src.__dict__:
    print key, getattr(src, key)

In [ ]:
if src.source_type == 'SimpleFaultSource':
    
    if src.use_slip == 1:
        
        # Fix upper seismogenic depth
        if not src.__dict__.has_key('upper_seismogenic_depth'):
            upper_seismogenic_depth = 0
        else:
            upper_seismogenic_depth = src.upper_seismogenic_depth
            
        # Scaling relationship
        if 'scalerel' in src.__dict__:
            scalerel = getattr(src, 'scalerel')
        else:
            scalerel = WC1994()

        # Compute area source
        surf = SimpleFaultSurface.from_fault_data(
                    fault_trace = src.trace, 
                    upper_seismogenic_depth = upper_seismogenic_depth, 
                    lower_seismogenic_depth = src.lower_seismogenic_depth, 
                    dip = src.dip, 
                    mesh_spacing = 2.5)
        flt_area = surf.get_area()

        # Median magnitude
        if 'mmax' in src.__dict__ and float(getattr(src, 'mmax')) > 0.0:
            median_magnitude = float(getattr(src, 'mmax'))
            print 'Maximum magnitude (from input)               : %.2f' % (median_magnitude)
        else:
            median_magnitude = scalrel.get_median_mag(flt_area, src.rake)
            std = scalrel.get_std_dev_area()
            mmax = median_magnitude + std * 2
            print 'Maximum magnitude (from scaling relationship): %.2f' % (median_magnitude)
         
        if median_magnitude > m_low+bin_width and src.slip_rate > 1e-10:
        
            # Get rates
            if hasattr(src, 'id_intersecting_area'):
                print 'Using parameters of area source ID           : %s' % (src.id_intersecting_area)
                b_gr = model.sources[src.id_intersecting_area].b_gr
            else:
                b_gr = model.default_bgr
            
            if 'coeff' in src.__dict__:
                coeff = float(getattr(src, 'coeff'))
                print 'Coeff parameter (from input)                 : %.2f' % (coeff)
            else:
                coeff = 1.0
                print 'Coeff parameter (default)                    : %.2f' % (coeff)
            
            rates = rates_for_double_truncated_mfd(flt_area, 
                                                   src.slip_rate*coeff, 
                                                   m_low, 
                                                   median_magnitude, 
                                                   b_gr=b_gr,
                                                   bin_width=bin_width)
            
            if any(rates):
                src.mfd = EvenlyDiscretizedMFD(m_low+bin_width/2, bin_width, rates)
                src.scalerel = scalrel
                print 'Computed MFD\n',src.mfd.get_annual_occurrence_rates()
            else:
                print 'MFD not defined: rates not defined'
                src.mfd = None
                src.scalerel = None                
        else:  
            print 'MFD not defined: mag from scaling relationship < mmin + bin_width'
            src.mfd = None
            src.scalerel = None
    else:
        print 'For this source we do not use slip'

model.sources[source_ids_list[0]] = src 
oqtkp.models[model_id] = model
oqtkp.save()